#Установка ClickHouse


In [1]:
!sudo apt-get install apt-transport-https ca-certificates dirmngr
!sudo apt-key adv --keyserver hkp://keyserver.ubuntu.com:80 --recv E0C56BD4

!echo "deb https://repo.clickhouse.tech/deb/stable/ main/" | sudo tee \
    /etc/apt/sources.list.d/clickhouse.list
!sudo apt-get update

!sudo apt-get install -y clickhouse-server clickhouse-client

!sudo service clickhouse-server start
!clickhouse-client

Reading package lists... Done
Building dependency tree       
Reading state information... Done
ca-certificates is already the newest version (20210119~18.04.1).
dirmngr is already the newest version (2.2.4-1ubuntu1.4).
The following NEW packages will be installed:
  apt-transport-https
0 upgraded, 1 newly installed, 0 to remove and 29 not upgraded.
Need to get 1,696 B of archives.
After this operation, 153 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 apt-transport-https all 1.6.12ubuntu0.2 [1,696 B]
Fetched 1,696 B in 0s (47.6 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend:

In [2]:
!pip install clickhouse-driver

     |████████████████████████████████| 481kB 11.9MB/s 


In [30]:
from clickhouse_driver import Client
client = Client(host='localhost')
client.execute('SHOW DATABASES')

[('db_movies',), ('default',), ('system',)]

# Загрузка датасетов

In [4]:
import pandas as pd
import numpy as np

In [14]:
df_ratings = pd.read_csv('drive/MyDrive/dataset/ratings.csv')

In [15]:
df_ratings['date'] = pd.to_datetime(df_ratings['timestamp'],unit='s')

In [16]:
df_ratings = df_ratings[['date','userId','movieId','rating']]
df_ratings.columns = ['date','userid','movieid','rating']

In [17]:
df_ratings.head()

,date,userid,movieid,rating
0,2009-12-14 02:52:24,1,31,2.5
1,2009-12-14 02:52:59,1,1029,3.0
2,2009-12-14 02:53:02,1,1061,3.0
3,2009-12-14 02:53:05,1,1129,2.0
4,2009-12-14 02:53:25,1,1172,4.0


In [19]:
df_ratings.isnull().sum()

date       0
userid     0
movieid    0
rating     0
dtype: int64

In [20]:
df_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100004 entries, 0 to 100003
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype         
---  ------   --------------   -----         
 0   date     100004 non-null  datetime64[ns]
 1   userid   100004 non-null  int64         
 2   movieid  100004 non-null  int64         
 3   rating   100004 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(2)
memory usage: 3.1 MB


In [53]:
usecols = ['id','title']
df_movies = pd.read_csv('drive/MyDrive/dataset/movies.csv',usecols=usecols)

In [54]:
df_movies.head()

,id,title
0,862,Toy Story
1,8844,Jumanji
2,15602,Grumpier Old Men
3,31357,Waiting to Exhale
4,11862,Father of the Bride Part II


In [55]:
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      45466 non-null  object
 1   title   45460 non-null  object
dtypes: object(2)
memory usage: 710.5+ KB


In [56]:
df_movies.id = pd.to_numeric(df_movies.id , errors='coerce').astype('Int32')

In [57]:
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      45463 non-null  Int32 
 1   title   45460 non-null  object
dtypes: Int32(1), object(1)
memory usage: 577.3+ KB


In [58]:
df_movies.isnull().sum()

id       3
title    6
dtype: int64

In [59]:
df_movies = df_movies.dropna()

In [60]:
df_movies.isnull().sum()

id       0
title    0
dtype: int64

# Добавление новой БД и запись значений

In [32]:
# client.execute('CREATE DATABASE db_movies')
client.execute('SHOW DATABASES')

[('db_movies',), ('default',), ('system',)]

In [33]:
client = Client(host='localhost', user='default', port='9000', database='db_movies')

In [34]:
client.execute('SHOW DATABASES')

[('db_movies',), ('default',), ('system',)]

In [ ]:
client.execute('DROP TABLE IF EXISTS ratings')

In [35]:
client.execute('CREATE TABLE ratings (date Date, userid Int32, movieid Int32, rating Float32  ) ENGINE = Memory')

[]

In [36]:
client.execute('SHOW TABLES FROM db_movies')

[('ratings',)]

In [42]:
client.execute("INSERT INTO ratings VALUES", df_ratings.to_dict('records'))

100004

In [43]:
client.execute('SELECT * FROM ratings LIMIT 5')

[(datetime.date(2009, 12, 14), 1, 31, 2.5),
 (datetime.date(2009, 12, 14), 1, 1029, 3.0),
 (datetime.date(2009, 12, 14), 1, 1061, 3.0),
 (datetime.date(2009, 12, 14), 1, 1129, 2.0),
 (datetime.date(2009, 12, 14), 1, 1172, 4.0)]

In [44]:
client.execute('DROP TABLE IF EXISTS movies')

[]

In [45]:
client.execute('CREATE TABLE movies (id Int32, title String ) ENGINE = Memory')

[]

In [46]:
client.execute('SHOW TABLES FROM db_movies')

[('movies',), ('ratings',)]

In [61]:
client.execute("INSERT INTO movies VALUES", df_movies.to_dict('records'))

45460

In [62]:
client.execute('SELECT * FROM movies LIMIT 5')

[(862, 'Toy Story'),
 (8844, 'Jumanji'),
 (15602, 'Grumpier Old Men'),
 (31357, 'Waiting to Exhale'),
 (11862, 'Father of the Bride Part II')]

# Запросы к БД

In [99]:
client.execute('SELECT r.*,  \
                      m.title \
                FROM  \
                    (SELECT ratings.movieid,  \
                            count(ratings.rating) as count,  \
                            avg(ratings.rating) as avg \
                    FROM ratings \
                    GROUP BY ratings.movieid  \
                    ORDER BY count(ratings.rating) DESC, \
                            avg(ratings.rating) DESC) as r  \
                    LEFT JOIN movies as m ON r.movieid = m.id \
                LIMIT 10')

[(356, 341, 4.05425219941349, ''),
 (296, 324, 4.256172839506172, 'Terminator 3: Rise of the Machines'),
 (318, 311, 4.487138263665595, 'The Million Dollar Hotel'),
 (593, 304, 4.1381578947368425, 'Solaris'),
 (260, 291, 4.221649484536083, 'The 39 Steps'),
 (480, 274, 3.7062043795620436, 'Monsoon Wedding'),
 (2571, 259, 4.183397683397684, ''),
 (1, 247, 3.8724696356275303, ''),
 (527, 244, 4.30327868852459, 'Once Were Warriors'),
 (589, 237, 4.006329113924051, '')]